Code extracted from nems/scripts/demo_script_2p.py

In [2]:
import os
import logging
import pandas as pd
import pickle
import sys
import numpy as np

from nems.gui.recording_browser import browse_recording, browse_context
import nems
import nems.initializers
import nems.priors
import nems.preprocessing as preproc
import nems.plots.api as nplt
import nems.modelspec as ms
import nems.xforms as xforms
import nems.utils
import nems.uri
import nems.recording as recording
from nems.signal import RasterizedSignal
from nems.fitters.api import scipy_minimize
import nems.db as nd

log = logging.getLogger(__name__)


[nems.configs.defaults INFO] Saving log messages to /tmp/nems/NEMS 2018-12-05 121831.log


here's the loading/parameter setting part

In [3]:
# ----------------------------------------------------------------------------
# LOAD AND FORMAT RECORDING DATA

# data file and results locations
# defined in nems/nems/configs/settings.py, which will override
# defaults in nems/nems/configs/defaults.py
results_dir = nems.get_setting('NEMS_RESULTS_DIR')
recordings_dir = nems.get_setting('NEMS_RECORDINGS_DIR')

# 2p data from Polley Lab at EPL
respfile = os.path.join(recordings_dir, 'data_nems_2p/neurons.csv')
stimfile = os.path.join(recordings_dir, 'data_nems_2p/stim_spectrogram.csv')
exptid = "POL001"
cellid = "POL001-080"
batch = 1  # define the group of data this belong to (eg, 1: A1, 2: AAF, etc)
load_command='nems.demo.loaders.load_polley_data'

# MODEL SPEC
# modelspecname = 'dlog_wcg18x1_stp1_fir1x15_lvl1_dexp1'
modelspecname = 'wc.9x1.g-fir.1x15-lvl.1'
meta = {'cellid': cellid, 'batch': batch, 'modelname': modelspecname}


In [20]:
from nems.demo.loaders import load_polley_data
r = load_polley_data(load_command=load_command,
                respfile=respfile,
                stimfile=stimfile,
                exptid=exptid,
                cellid=cellid)

In [25]:
print(r.keys())
print(r['stim'].shape)

dict_keys(['fs', 'resp', 'stim', 'stim_labels', 'resp_labels'])
(9, 85051)


In [26]:
del r

In [27]:
xfspec = []
xfspec.append(['nems.xforms.load_recording_wrapper',
               {'load_command': load_command,
                'respfile': respfile,
                'stimfile': stimfile,
                'exptid': exptid,
                'cellid': cellid}])

# reserve 10% of the data for validation
xfspec.append(['nems.xforms.split_at_time', {'valfrac': 0.1}])

xfspec.append(['nems.xforms.init_from_keywords',
               {'keywordstring': modelspecname, 'meta': meta}])

xfspec.append(['nems.xforms.fit_basic_init', {}])
xfspec.append(['nems.xforms.fit_basic', {}])
xfspec.append(['nems.xforms.predict',    {}])
# xfspec.append(['nems.xforms.add_summary_statistics',    {}])
xfspec.append(['nems.analysis.api.standard_correlation', {},
               ['est', 'val', 'modelspec', 'rec'], ['modelspec']])
xfspec.append(['nems.xforms.plot_summary',    {}])


In [28]:
xfspec

[['nems.xforms.load_recording_wrapper',
  {'load_command': 'nems.demo.loaders.load_polley_data',
   'respfile': '/Users/svd/python/nems/recordings/data_nems_2p/neurons.csv',
   'stimfile': '/Users/svd/python/nems/recordings/data_nems_2p/stim_spectrogram.csv',
   'exptid': 'POL001',
   'cellid': 'POL001-080'}],
 ['nems.xforms.split_at_time', {'valfrac': 0.1}],
 ['nems.xforms.init_from_keywords',
  {'keywordstring': 'wc.9x1.g-fir.1x15-lvl.1',
   'meta': {'cellid': 'POL001-080',
    'batch': 1,
    'modelname': 'wc.9x1.g-fir.1x15-lvl.1'}}],
 ['nems.xforms.fit_basic_init', {}],
 ['nems.xforms.fit_basic', {}],
 ['nems.xforms.predict', {}],
 ['nems.analysis.api.standard_correlation',
  {},
  ['est', 'val', 'modelspec', 'rec'],
  ['modelspec']],
 ['nems.xforms.plot_summary', {}]]

In [ ]:
# actually do the fit
ctx, log_xf = xforms.evaluate(xfspec)

# shorthand for:
# ctx = {}
# for xfa in xfspec:
#     ctx = xforms.evaluate_step(xfa, ctx)


[nems.xforms INFO] Evaluating: nems.xforms.load_recording_wrapper
[nems.xforms INFO] Loading cached file /Users/svd/python/nems/recordings/POL001_86e767f767be0796a601a03ba9fa253d474ce19a.tgz


In [ ]:
# testing new plotting system
modelspec = ctx['modelspec']
modelspec.quickplot(ctx['val'])

In [ ]:
# ----------------------------------------------------------------------------
# SAVE YOUR RESULTS

# save results to file
destination = os.path.join(results_dir, str(batch), xforms.get_meta(ctx)['cellid'],
                           ms.get_modelspec_longname(ctx['modelspec']))
log.info('Saving modelspec(s) to {0} ...'.format(destination))
xforms.save_analysis(destination,
                     recording=ctx['rec'],
                     modelspec=ctx['modelspec'],
                     xfspec=xfspec,
                     figures=ctx['figures'],
                     log=log_xf)

# save summary of results to a database
log.info('Saving metadata to db  ...')
modelspec = ctx['modelspec']
modelspec.meta()['modelpath'] = destination
modelspec.meta()['figurefile'] = destination + 'figure.0000.png'
nd.update_results_table(modelspec)
    

In [ ]:
# reload using:
"""
# repeated from above
import nems.db as nd
import nems.xforms as xforms

# pick your cell/batch/model
cellid = "POL001-080"
batch = 1  # define the group of data this belong to (eg, 1: A1, 2: AAF, etc)
modelspecname = 'wc.9x1.g-fir.1x15-lvl.1'

# find the results in the database
d=nd.get_results_file(batch=batch, cellids=[cellid], modelnames=[modelspecname])
d.loc[0]
filepath = d['modelpath'][0] + '/'

# load and display results
xfspec, ctx = xforms.load_analysis(filepath, eval_model=True)
"""
